Emotion Detection using Active Learning.

Copyright (C) 2022  Rigved Rakshit

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU Affero General Public License as published
by the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU Affero General Public License for more details.

You should have received a copy of the GNU Affero General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
# Introduction

This Jupyter notebook will extract the data from the emotion annotations obtained from Qualtrics and add them to a new dataset built on top of the original dataset released by Ferracane et al. (2021).

# Code

In [1]:
# Import the required libraries

import os
import nltk
import numpy as np
import pandas as pd
from IPython.core.display import HTML
from IPython.display import display

In [2]:
# Set options for pandas display
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_seq_item', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 2)

In [3]:
# Set variables
emotion_annotated_dataset_path = '/home/rigved/Documents/USA/Universities/RIT/Courses/2022/Spring/DSCI_602_Applied_Data_Science_II/Data/CSVs'

In [4]:
# Correct errors in the survey questionaire
!sed -i 's/For for the first selected emotion, please indicate the text spans in the /For the first selected emotion, please indicate the text spans in the /g' {emotion_annotated_dataset_path}/*

In [5]:
# Load the emotion dataset
emotion_annotated_data = {}

for root, _, files in os.walk(emotion_annotated_dataset_path):
    for filename in files:
        if filename.startswith('.'):
            continue
        
        # Find the hearing ID and the subject ID from the filenames
        hearing_id = (filename.split('.')[0]).split('_')[-2]
        subject_id = (filename.split('.')[0]).split('_')[-1]
        
        df = pd.read_csv(os.path.join(emotion_annotated_dataset_path, filename))
        
        if hearing_id not in emotion_annotated_data.keys():
            emotion_annotated_data[hearing_id] = {}
        
        emotion_annotated_data[hearing_id][subject_id] = df

In [6]:
# Display the loaded dataset

emotion_annotated_data['CHRG-114hhrg20722']['bKNm']

StartDate  \
0                                            Start Date   
1  {"ImportId":"startDate","timeZone":"America/Denver"}   
2                                   2022-03-23 12:56:00   

                                              EndDate                 Status  \
0                                            End Date          Response Type   
1  {"ImportId":"endDate","timeZone":"America/Denver"}  {"ImportId":"status"}   
2                                 2022-03-24 05:29:16             IP Address   

                  Progress    Duration (in seconds)                 Finished  \
0                 Progress    Duration (in seconds)                 Finished   
1  {"ImportId":"progress"}  {"ImportId":"duration"}  {"ImportId":"finished"}   
2                      100                    59593                     True   

                                              RecordedDate  \
0                                            Recorded Date   
1  {"ImportId":"recordedDate","timeZone":"America/Denver"}   
2                                      2022-03-24 05:29:21   

                 ResponseId                 DistributionChannel  \
0               Response ID                Distribution Channel   
1  {"ImportId":"_recordId"}  {"ImportId":"distributionChannel"}   
2         R_1IL6BLRngrQ3DPP                           anonymous   

                  UserLanguage                       subject_id  \
0                User Language  Subject ID (found in the email)   
1  {"ImportId":"userLanguage"}         {"ImportId":"QID4_TEXT"}   
2                           EN                             bKNm   

                                                                                                                                                                                                                                             q0_emotions  \
0  First, you'll answer questions about the question. You'll select the emotion conveyed in the question.\n\n\n\nSelect the Disapproval and Annoyance emotions for the example question. (Tip: Use Ctrl+Click or Cmd+Click to select multiple emotions).   
1                                                                                                                                                                                                                                   {"ImportId":"QID47"}   
2                                                                                                                                                                                                                                  Annoyance,Disapproval   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     q0_emotion2_t_1  \
0  Next, you'll select the text spans that indicate the first selected emotion (Annoyance in this case). Note that this is the first selected emotion by order in the question emotion list above.\n\n \nSelect the words "annoyed" for the Annoyance emotion in the below text. You can click on a word, and then click+drag your cursor to select multiple words. After selecting a text passage, you must click on the "Emotion 1" label for your answer to be entered. It then shows with a green background.\n\n\n\nYou can deselect text spans by selecting the highlighted text and clicking on the "Remove" button.\n\n \nIf there is too m

In [7]:
# Load the associated questions and responses and combine it with the emotion dataset

complete_data_with_duplicates = pd.read_csv('/home/rigved/repos/rit/DSCI-601/subjective_discourse/data/gold/expanded_with_features_annotated_questions_responses_gold.csv')
complete_data_without_duplicates = complete_data_with_duplicates.drop_duplicates(subset='qa_index')

merged_emotion_annotated_data_list = list()

for hearing_id in sorted(emotion_annotated_data.keys()):
    for subject_id in sorted(emotion_annotated_data[hearing_id]):
        question_response_iterator = complete_data_without_duplicates[complete_data_without_duplicates['hearing_id'] == hearing_id][['q_text', 'r_text']].itertuples(index=False)
        question_response_index = 0
        is_question = True
        
        for _, column_data in emotion_annotated_data[hearing_id][subject_id].iteritems():
            if column_data.iloc[0].startswith('Which emotion(s) did the congressperson convey in the question? Please select all that apply (max 2).'):
                # We are the start of a new question-response block
                is_question = True
                question_response_index += 1
                data = {
                    'hearing_id': hearing_id,
                    'subject_id': subject_id,
                    'question-response_number': question_response_index,
                    'question': '',
                    'response': '',
                    'q_emotion1': '',
                    'q_emotion2': '',
                    'q_emotion1_text': '',
                    'q_emotion1_intensity': '',
                    'q_emotion1_intensity_text': '',
                    'q_emotion1_confidence': '',
                    'q_emotion2_text': '',
                    'q_emotion2_intensity': '',
                    'q_emotion2_intensity_text': '',
                    'q_emotion2_confidence': '',
                    'r_emotion1': '',
                    'r_emotion2': '',
                    'r_emotion1_text': '',
                    'r_emotion1_intensity': '',
                    'r_emotion1_intensity_text': '',
                    'r_emotion1_confidence': '',
                    'r_emotion2_text': '',
                    'r_emotion2_intensity': '',
                    'r_emotion2_intensity_text': '',
                    'r_emotion2_confidence': '',
                    'a_emotion1': '',
                    'a_emotion2': ''
                }
                
                # Load the next question and response pairs
                if question_response_index <= 50:
                    question_response_text = next(question_response_iterator)
                    data['question'] = question_response_text[0].replace('    ', ' ').replace('   ', ' ').replace('``', '"').replace("''", '"').strip()
                    data['response'] = question_response_text[1].replace('    ', ' ').replace('   ', ' ').replace('``', '"').replace("''", '"').strip()
                
                # Used to keep track of breaks in highlighted utterances
                q1_word_index = -1
                q1_intensity_word_index = -1
                q2_word_index = -1
                q2_intensity_word_index = -1
                r1_word_index = -1
                r1_intensity_word_index = -1
                r2_word_index = -1
                r2_intensity_word_index = -1
                
                # Extract the emotion labels
                if not pd.isnull(column_data.iloc[2]):
                    q_emotions = column_data.iloc[2].split(',')
                
                    if len(q_emotions) == 2:
                        data['q_emotion2'] = q_emotions[1]
                    
                    data['q_emotion1'] = q_emotions[0]
            elif column_data.iloc[0].startswith('For the first selected emotion, please indicate the text spans in the question that helped you indicate the emotion. - Emotion 1'):
                # We are capturing the text that indicates emotion 1
                is_question = True
                
                if not pd.isnull(column_data.iloc[2]):
                    for emotion_text in column_data.iloc[2].split(','):
                        if emotion_text == '':
                            data['q_emotion1_text'] += ','
                        elif ' ' in emotion_text:
                            curr_q_word_index = int((emotion_text.split(' ')[0]).split(':')[0])
                            
                            if q1_word_index != -1 and curr_q_word_index != (q1_word_index + 1):
                                data['q_emotion1_text'] += ' [SEP]'
                            
                            data['q_emotion1_text'] += ' ' + emotion_text.split(' ')[1]
                            q1_word_index = curr_q_word_index
                
                    data['q_emotion1_text'] += ' [SEP]'
            elif column_data.iloc[0].startswith('Using the slider, indicate the intensity for the first selected emotion from 1 to 10.') and is_question:
                # We are capturing the intensity of emotion 1 in the question
                if not pd.isnull(column_data.iloc[2]):
                    data['q_emotion1_intensity'] = column_data.iloc[2]
            elif column_data.iloc[0].startswith('Please indicate the text spans in the question that helped you indicate the intensity of the first selected emotion. - Emotion 1'):
                # We are capturing the text that indicates the intensity emotion 1
                is_question = True
                
                if not pd.isnull(column_data.iloc[2]):
                    for emotion_text in column_data.iloc[2].split(','):
                        if emotion_text == '':
                            data['q_emotion1_intensity_text'] += ','
                        elif ' ' in emotion_text:
                            curr_q_intensity_word_index = int((emotion_text.split(' ')[0]).split(':')[0])
                            
                            if q1_intensity_word_index != -1 and curr_q_intensity_word_index != (q1_intensity_word_index + 1):
                                data['q_emotion1_intensity_text'] += ' [SEP]'
                            
                            data['q_emotion1_intensity_text'] += ' ' + emotion_text.split(' ')[1]
                            q1_intensity_word_index = curr_q_intensity_word_index
                
                    data['q_emotion1_intensity_text'] += ' [SEP]'
            elif column_data.iloc[0].startswith('For the first selected emotion, please indicate how confident you are from 0% to 100% that the congressperson conveyed that first selected emotion in the question.'):
                # We are capturing the confidence level of the first annotated emotion
                is_question = True
                
                if not pd.isnull(column_data.iloc[2]):
                    data['q_emotion1_confidence'] = column_data.iloc[2]
            elif column_data.iloc[0].startswith('For the second selected emotion, please indicate the text spans in the question that helped you indicate the emotion. - Emotion 2'):
                # We are capturing the text that indicates emotion 2
                is_question = True
                
                if not pd.isnull(column_data.iloc[2]):
                    for emotion_text in column_data.iloc[2].split(','):
                        if emotion_text == '':
                            data['q_emotion2_text'] += ','
                        elif ' ' in emotion_text:
                            curr_q_word_index = int((emotion_text.split(' ')[0]).split(':')[0])
                            
                            if q2_word_index != -1 and curr_q_word_index != (q2_word_index + 1):
                                data['q_emotion2_text'] += ' [SEP]'
                            
                            data['q_emotion2_text'] += ' ' + emotion_text.split(' ')[1]
                            q2_word_index = curr_q_word_index
                
                    data['q_emotion2_text'] += ' [SEP]'
            elif column_data.iloc[0].startswith('Using the slider, indicate the intensity for the second selected emotion from 1 to 10.') and is_question:
                # We are capturing the intensity of emotion 2
                if not pd.isnull(column_data.iloc[2]):
                    data['q_emotion2_intensity'] = column_data.iloc[2]
            elif column_data.iloc[0].startswith('Please indicate the text spans in the question that helped you indicate the intensity of the second selected emotion. - Emotion 2'):
                # We are capturing the text that indicates the intensity emotion 2
                is_question = True
                
                if not pd.isnull(column_data.iloc[2]):
                    for emotion_text in column_data.iloc[2].split(','):
                        if emotion_text == '':
                            data['q_emotion2_intensity_text'] += ','
                        elif ' ' in emotion_text:
                            curr_q_intensity_word_index = int((emotion_text.split(' ')[0]).split(':')[0])
                            
                            if q2_intensity_word_index != -1 and curr_q_intensity_word_index != (q2_intensity_word_index + 1):
                                data['q_emotion2_intensity_text'] += ' [SEP]'
                            
                            data['q_emotion2_intensity_text'] += ' ' + emotion_text.split(' ')[1]
                            q2_intensity_word_index = curr_q_intensity_word_index
                
                    data['q_emotion2_intensity_text'] += ' [SEP]'
            elif column_data.iloc[0].startswith('For the second selected emotion, please indicate how confident you are from 0% to 100% that the congressperson conveyed that second selected emotion in the question.'):
                # We are capturing the confidence level of the second annotated emotion
                is_question = True
                
                if not pd.isnull(column_data.iloc[2]):
                    data['q_emotion2_confidence'] = column_data.iloc[2]
            elif column_data.iloc[0].startswith('Which emotion(s) did the witness convey in the response? Please select all that apply (max 2).'):
                # We are now capturing the data for the response
                is_question = False
                
                # Extract the emotion labels
                if not pd.isnull(column_data.iloc[2]):
                    r_emotions = column_data.iloc[2].split(',')
                
                    if len(r_emotions) == 2:
                        data['r_emotion2'] = r_emotions[1]
                    
                    data['r_emotion1'] = r_emotions[0]
            elif column_data.iloc[0].startswith('For for the first selected emotion, please indicate the text spans in the response that helped you indicate the emotion. - Emotion 1'):
                # We are capturing the text that indicates emotion 1
                is_question = False
                
                if not pd.isnull(column_data.iloc[2]):
                    for emotion_text in column_data.iloc[2].split(','):
                        if emotion_text == '':
                            data['r_emotion1_text'] += ','
                        elif ' ' in emotion_text:
                            curr_r_word_index = int((emotion_text.split(' ')[0]).split(':')[0])
                            
                            if r1_word_index != -1 and curr_r_word_index != (r1_word_index + 1):
                                data['r_emotion1_text'] += ' [SEP]'
                            
                            data['r_emotion1_text'] += ' ' + emotion_text.split(' ')[1]
                            r1_word_index = curr_r_word_index
                
                    data['r_emotion1_text'] += ' [SEP]'
            elif column_data.iloc[0].startswith('Using the slider, indicate the intensity for the first selected emotion from 1 to 10.') and not is_question:
                # We are capturing the intensity of emotion 1 in the response
                if not pd.isnull(column_data.iloc[2]):
                    data['r_emotion1_intensity'] = column_data.iloc[2]
            elif column_data.iloc[0].startswith('Please indicate the text spans in the response that helped you indicate the intensity of the first selected emotion. - Emotion 1'):
                # We are capturing the text that indicates the intensity emotion 1
                is_question = False
                
                if not pd.isnull(column_data.iloc[2]):
                    for emotion_text in column_data.iloc[2].split(','):
                        if emotion_text == '':
                            data['r_emotion1_intensity_text'] += ','
                        elif ' ' in emotion_text:
                            curr_r_intensity_word_index = int((emotion_text.split(' ')[0]).split(':')[0])
                            
                            if r1_intensity_word_index != -1 and curr_r_intensity_word_index != (r1_intensity_word_index + 1):
                                data['r_emotion1_intensity_text'] += ' [SEP]'
                            
                            data['r_emotion1_intensity_text'] += ' ' + emotion_text.split(' ')[1]
                            r1_intensity_word_index = curr_r_intensity_word_index
                
                    data['r_emotion1_intensity_text'] += ' [SEP]'
            elif column_data.iloc[0].startswith('For the first selected emotion, please indicate how confident you are from 0% to 100% that the witness conveyed that first selected emotion in the response.'):
                # We are capturing the confidence level of the first annotated emotion
                is_question = False
                
                if not pd.isnull(column_data.iloc[2]):
                    data['r_emotion1_confidence'] = column_data.iloc[2]
            elif column_data.iloc[0].startswith('For for the second selected emotion, please indicate the text spans in the response that helped you indicate the emotion. - Emotion 2'):
                # We are capturing the text that indicates emotion 2
                is_question = False
                
                if not pd.isnull(column_data.iloc[2]):
                    for emotion_text in column_data.iloc[2].split(','):
                        if emotion_text == '':
                            data['r_emotion2_text'] += ','
                        elif ' ' in emotion_text:
                            curr_r_word_index = int((emotion_text.split(' ')[0]).split(':')[0])
                            
                            if r2_word_index != -1 and curr_r_word_index != (r2_word_index + 1):
                                data['r_emotion2_text'] += ' [SEP]'
                            
                            data['r_emotion2_text'] += ' ' + emotion_text.split(' ')[1]
                            r2_word_index = curr_r_word_index
                
                    data['r_emotion2_text'] += ' [SEP]'
            elif column_data.iloc[0].startswith('Using the slider, indicate the intensity for the second selected emotion from 1 to 10.') and not is_question:
                # We are capturing the intensity of emotion 2 in the response
                if not pd.isnull(column_data.iloc[2]):
                    data['r_emotion2_intensity'] = column_data.iloc[2]
            elif column_data.iloc[0].startswith('Please indicate the text spans in the response that helped you indicate the intensity of the second selected emotion. - Emotion 2'):
                # We are capturing the text that indicates the intensity emotion 2
                is_question = False
                
                if not pd.isnull(column_data.iloc[2]):
                    for emotion_text in column_data.iloc[2].split(','):
                        if emotion_text == '':
                            data['r_emotion2_intensity_text'] += ','
                        elif ' ' in emotion_text:
                            curr_r_intensity_word_index = int((emotion_text.split(' ')[0]).split(':')[0])
                            
                            if r2_intensity_word_index != -1 and curr_r_intensity_word_index != (r2_intensity_word_index + 1):
                                data['r_emotion2_intensity_text'] += ' [SEP]'
                            
                            data['r_emotion2_intensity_text'] += ' ' + emotion_text.split(' ')[1]
                            r2_intensity_word_index = curr_r_intensity_word_index
                
                    data['r_emotion2_intensity_text'] += ' [SEP]'
            elif column_data.iloc[0].startswith('For the second selected emotion, please indicate how confident you are from 0% to 100% that the witness conveyed that second selected emotion in the response.'):
                # We are capturing the confidence level of the second annotated emotion
                is_question = False
                
                if not pd.isnull(column_data.iloc[2]):
                    data['r_emotion2_confidence'] = column_data.iloc[2]
            elif column_data.iloc[0].startswith('How did this question-answer pair make you feel? Please select all that apply.'):
                # We are capturing the emotion felt by the annotator after reading the question-response pair
                is_question = False
                
                if not pd.isnull(column_data.iloc[2]):
                    a_emotions = column_data.iloc[2].split(',')
                
                    if len(a_emotions) == 2:
                        data['a_emotion2'] = a_emotions[1]
                    
                    data['a_emotion1'] = a_emotions[0]
                
                # We are at the end of this question-response pair
                # So, remove extra tokens at the begining and the end,
                # and append the data to the list
                data['q_emotion1_text'] = data['q_emotion1_text'].strip('[SEP]').strip()
                data['q_emotion1_intensity_text'] = data['q_emotion1_intensity_text'].strip('[SEP]').strip()
                data['q_emotion2_text'] = data['q_emotion2_text'].strip('[SEP]').strip()
                data['q_emotion2_intensity_text'] = data['q_emotion2_intensity_text'].strip('[SEP]').strip()
                data['r_emotion1_text'] = data['r_emotion1_text'].strip('[SEP]').strip()
                data['r_emotion1_intensity_text'] = data['r_emotion1_intensity_text'].strip('[SEP]').strip()
                data['r_emotion2_text'] = data['r_emotion2_text'].strip('[SEP]').strip()
                data['r_emotion2_intensity_text'] = data['r_emotion2_intensity_text'].strip('[SEP]').strip()
                
                merged_emotion_annotated_data_list.append(data)

# Create the final merged Pandas DataFrame
merged_emotion_annotated_data = pd.DataFrame(
    merged_emotion_annotated_data_list,
    columns=[
        'hearing_id',
        'subject_id',
        'question-response_number',
        'question',
        'response',
        'q_emotion1',
        'q_emotion2',
        'q_emotion1_text',
        'q_emotion1_intensity',
        'q_emotion1_intensity_text',
        'q_emotion1_confidence',
        'q_emotion2_text',
        'q_emotion2_intensity',
        'q_emotion2_intensity_text',
        'q_emotion2_confidence',
        'r_emotion1',
        'r_emotion2',
        'r_emotion1_text',
        'r_emotion1_intensity',
        'r_emotion1_intensity_text',
        'r_emotion1_confidence',
        'r_emotion2_text',
        'r_emotion2_intensity',
        'r_emotion2_intensity_text',
        'r_emotion2_confidence',
        'a_emotion1',
        'a_emotion2'
])

# Save this merged dataframe to disk
merged_emotion_annotated_data.to_csv(
    '/home/rigved/Documents/USA/Universities/RIT/Courses/2022/Spring/DSCI_602_Applied_Data_Science_II/Data/emotion_data_annotated_by_humans.csv',
    index=False
)

In [8]:
merged_emotion_annotated_data.loc[merged_emotion_annotated_data['subject_id'] == 'bKNm'].head(10)

,hearing_id,subject_id,question-response_number,question,response,q_emotion1,q_emotion2,q_emotion1_text,q_emotion1_intensity,q_emotion1_intensity_text,q_emotion1_confidence,q_emotion2_text,q_emotion2_intensity,q_emotion2_intensity_text,q_emotion2_confidence,r_emotion1,r_emotion2,r_emotion1_text,r_emotion1_intensity,r_emotion1_intensity_text,r_emotion1_confidence,r_emotion2_text,r_emotion2_intensity,r_emotion2_intensity_text,r_emotion2_confidence,a_emotion1,a_emotion2
52,CHRG-114hhrg20722,bKNm,1,"Secretary Clinton stated that she did not email any classified material, and Director Comey stated there was classified material emailed. Do you agree with Director Comey's conclusion about that?","Again, I would have to refer you to Director Comey's statements for the basis for his recommendations.",Interest,,Do you agree with Director Comey's conclusion about that?,5,,20,,,,,Annoyance,,,3,"Again,",80,,,,,Boredom,
53,CHRG-114hhrg20722,bKNm,2,Director Comey stated that there is evidence of potential violations of the statutes regarding the handling of classified information. Do you agree with Director Comey's statement?,"Again, I would refer you to Director Comey for any further explanation as to the basis for his recommendations. The recommendation that I received from the team, including Director Comey----",Interest,,Do you agree with Director Comey's statement?,3,,20,,,,,Annoyance,,,5,"Again, [SEP] The recommendation that I received from the team, including Director Comey----",99,,,,,Boredom,
54,CHRG-114hhrg20722,bKNm,3,"Let me ask you one final question that does not regard the specific facts with regard to Secretary Clinton, but Director Comey said that there was not clear evidence that Secretary Clinton or her colleagues intended to violate laws governing the handling of classified information. My question for you is, is intent to violate the law a requirement under 18 U.S.C. section 793(f)?","Well, Congressman, I think the statutes that were considered here speak for themselves. To answer further would require a discussion of the facts and the analysis of this matter, which, as I've indicated, I'm not in a position to provide at the time. Again, I refer you to Director Comey's discussion for that. As I've indicated, the team reviewed this matter, and it was a unanimous team decision.",Interest,,,2,,20,,,,,Annoyance,,,7,"Well, Congressman, [SEP] speak for themselves. [SEP] which, as I've indicated, [SEP] Again, [SEP] As I've indicated,",100,,,,,Boredom,
55,CHRG-114hhrg20722,bKNm,4,"And you made a decision, following their recommendation to you, that you were not going to prosecute and the matter was closed. Is that correct?","I made the decision some time ago that I would accept the recommendation of that team and was awaiting that recommendation. When I received it, there was no basis not to accept it. And, again, I reiterate my pride and faith in them.",Interest,,"you [SEP] you, [SEP] you [SEP] Is that correct?",4,,20,,,,,Acceptance,,,8,"made the decision some time ago [SEP] no basis not to accept it. [SEP] again, I reiterate my pride and faith",100,,,,,Boredom,
56,CHRG-114hhrg20722,bKNm,5,"Thank you. Thank you for being here again, Attorney General, and thank you very much for your frank and candid discussion with us that is now taking place. I'm looking for answers and views of some events that I'm going to string together and ask you to discuss as far as you can in an appropriate manner. Baton Rouge, Louisiana, police shot and killed Alton Sterling. Video shows that he was shot while being pinned to the ground by two officers. Outside of Minneapolis, police shot and killed Philando Castile at what should have been a routine traffic stop. He was armed, but reports suggest that he repeatedly told police that he had a valid permit for the weapon. In Dallas, a gunman killed five police officers and wounded seven others in what appeared to be a well-planned attack--this terrible act in the middle of an otherwise peaceful protest 

# References

E. Ferracane, G. Durrett, J.J. Li, and K. Erk. 2021. Did they answer? subjective acts and intents in conversational discourse. In Proceedings of the 2021 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, pages 1626–1644, Online. Association for Computational Linguistics.